In [1]:
import os
import time

import tools
import benchmarks as bm
from poseEvaluation import PoseErrorEvaluator
from evo.core import metrics

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib tk

import copy


[DEBUG][2025-06-17 11:36:19,859][log.configure_logging():113]
System info:
Python 3.13.2
Linux-6.2.0-060200-generic-x86_64-with-glibc2.31
i3t-sigma@I3T-Sigma



In [ ]:
benchmark_list = ["XREVA"]
trials = ['data0']

root_dir = "../Datasets"
os.chdir(root_dir)
root_dir = "."

In [ ]:
datadict = {} 
# motion-visual-speed: 3-level dict
# under which: 
#   "S1_raw": df_combined_csv
#   "S2_raw": df_combined_csv
#   "RPE_rmse": {"AVP": rmse, "MQ3": rmse, ... ,"ML2": rmse}
#   "RPE_std": {"AVP": std, "MQ3": std, ... ,"ML2": std}
#   "APE_rmse": {"AVP": rmse, "MQ3": rmse, ... ,"ML2": rmse}
#   "APE_std": {"AVP": std, "MQ3": std, ... ,"ML2": std}
#   "RPE": {"AVP": [], "MQ3": [], ... ,"ML2": []}
#   "APE": {"AVP": [], "MQ3": [], ... ,"ML2": []}
motions = ['Inspect', 'Patrol', 'Rotate', 'Shift']
visuals = ['Featurerich', 'Featureless']
speeds = ['50', '75']
for motion in motions:
    datadict[motion] = {}
    for visual in visuals:
        datadict[motion][visual] = {}
        for speed in speeds:
            datadict[motion][visual][speed] = {}
            datadict[motion][visual][speed]["Correlation"] = {}


In [ ]:

def load_trajectory_dict(Set="S1"):
    global datadict
    failed_trials = []
    for i, benchmark in enumerate(benchmark_list):
        benchmarkClass = bm.benchmark_factory[benchmark]
        benchmarkObject = benchmarkClass(root_dir)
        benchmarkObject.generate_script(Set)
        scriptList = benchmarkObject.get_script()

        check_traj_dict = {}
        for j, scriptDict in enumerate(scriptList):
            benchmark = scriptDict["benchmark"]
            trajectory = scriptDict["trajectory"]
            check_traj_dict[trajectory] = 0
        
        for j, scriptDict in enumerate(scriptList):
            benchmark = scriptDict["benchmark"]
            trajectory = scriptDict["trajectory"]
            #benchmarkObject.copy_ground_truth_traj(scriptDict["trajectory"])
            trial = scriptDict["trial"]

            combined_csv_path = "{}/{}/{}/merge_result_filled.csv".format(root_dir, trajectory, trial)
            try:
                df_traj_combined = pd.read_csv(combined_csv_path, sep=",")
            except FileNotFoundError:
                #print("File not found: {}/{}".format(trajectory, trial))
                failed_trials.append("{}/{}".format(trajectory, trial))
                continue

            check_traj_dict[trajectory] += 1

            configuration = trajectory.split("_")
            group, motion, visual, speed = configuration[0], configuration[1], configuration[2], configuration[3]

            # Rename
            if motion == "Petrol":
                motion = "Patrol"
            if motion == "Rotation":
                motion = "Rotate"
            if motion == "Side":
                motion = "Shift"
            
            assert motion in ['Inspect', 'Patrol', 'Rotate', 'Shift'], "Invalid motion: {}".format(motion)
            assert visual in ['Featurerich', 'Featureless'], "Invalid visual: {}".format(visual)
            assert speed in ['50', '75'], "Invalid speed: {}".format(speed)

            print("Group: {}, Motion: {}, Visual: {}, Speed: {}".format(group, motion, visual, speed))
            if Set == "S1":
                datadict[motion][visual][speed]["S1_raw"] = df_traj_combined
            else:
                datadict[motion][visual][speed]["S2_raw"] = df_traj_combined

    #print check traj that are 0
    #print("Check traj dict:")
    for key, value in check_traj_dict.items():
        if value == 0:
            print("Invalid trajectory: {}".format(key))
    return failed_trials
        

In [5]:
root_dir = "../Datasets"
os.chdir(root_dir)
root_dir = "."

In [ ]:
failed_trials = load_trajectory_dict("S1")

In [ ]:
failed_trials = load_trajectory_dict('S2')

## Start Preprocessing

In [8]:
# Function to find maximum empty rows
def max_empty_rows(df):
    max_empty = 0
    for col in df.columns:
        empty_count = 0
        for value in df[col]:
            if pd.isna(value):
                empty_count += 1
            else:
                break
        max_empty = max(max_empty, empty_count)
    return max_empty

In [ ]:
for motion in motions:
    for visual in visuals:
        for speed in speeds:
            print("Processing motion: {}, visual: {}, speed: {}".format(motion, visual, speed))
            df_combined_S1 = copy.deepcopy(datadict[motion][visual][speed]["S1_raw"])
            df_combined_S2 = copy.deepcopy(datadict[motion][visual][speed]["S2_raw"])

            # Remove the first nan rows
            max_empty_S1 = max_empty_rows(df_combined_S1)
            max_empty_S2 = max_empty_rows(df_combined_S2)
            print("{}-{}-{}: \t \t Max empty rows S1: {}-{} \t S2: {}-{}".format(motion, visual, speed, 
                                                                         max_empty_S1, 
                                                                         str(round(max_empty_S1*100/len(df_combined_S1),1))+"%",
                                                                         max_empty_S2, 
                                                                         str(round(max_empty_S2*100/len(df_combined_S2),1))+"%"))

            datadict[motion][visual][speed]["S1_trim"] = df_combined_S1.iloc[max_empty_S1:].reset_index(drop=True)
            datadict[motion][visual][speed]["S2_trim"] = df_combined_S2.iloc[max_empty_S2:].reset_index(drop=True)


Processing motion: Inspect, visual: Featurerich, speed: 50
Inspect-Featurerich-50: 	 	 Max empty rows S1: 304-5.0% 	 S2: 417-7.8%
Processing motion: Inspect, visual: Featurerich, speed: 75
Inspect-Featurerich-75: 	 	 Max empty rows S1: 308-5.5% 	 S2: 95-2.9%
Processing motion: Inspect, visual: Featureless, speed: 50
Inspect-Featureless-50: 	 	 Max empty rows S1: 298-3.4% 	 S2: 386-5.5%
Processing motion: Inspect, visual: Featureless, speed: 75
Inspect-Featureless-75: 	 	 Max empty rows S1: 407-7.1% 	 S2: 165-4.8%
Processing motion: Patrol, visual: Featurerich, speed: 50
Patrol-Featurerich-50: 	 	 Max empty rows S1: 226-3.5% 	 S2: 112-3.1%
Processing motion: Patrol, visual: Featurerich, speed: 75
Patrol-Featurerich-75: 	 	 Max empty rows S1: 264-5.8% 	 S2: 131-4.1%
Processing motion: Patrol, visual: Featureless, speed: 50
Patrol-Featureless-50: 	 	 Max empty rows S1: 315-5.0% 	 S2: 183-4.9%
Processing motion: Patrol, visual: Featureless, speed: 75
Patrol-Featureless-75: 	 	 Max empty ro

In [12]:
def device_rename(name):
    if name == "RPE":
        return "ORB3"
    elif name == "APE":
        return "ORB3"
    else:
        name = name.split("_")[0]
        if name == "AppleVisionPro":
            return "AVP"
        elif name == "MetaQuest3":
            return "MQ3"
        elif name == "XReal2Ultra":
            return "XR2U"
        elif name == "Hololens2":
            return "HL2"
        elif name == "MagicLeap2":
            return "ML2"
        else:
            return name

# Reverse the device name
def device_rename_reverse(name, ErrorType="RPE"):
    if ErrorType == "RPE":
        if name == "ORB3":
            return "RPE"
        else:
            if name == "AVP":
                return "AppleVisionPro"+"_RPE"
            elif name == "MQ3":
                return "MetaQuest3"+"_RPE"
            elif name == "XR2U":
                return "XReal2Ultra"+"_RPE"
            elif name == "HL2":
                return "Hololens2"+"_RPE"
            elif name == "ML2":
                return "MagicLeap2"+"_RPE"
            else:
                return name
    elif ErrorType == "APE":
        if name == "ORB3":
            return "APE"
        else:
            if name == "AVP":
                return "AppleVisionPro"+"_APE"
            elif name == "MQ3":
                return "MetaQuest3"+"_APE"
            elif name == "XR2U":
                return "XReal2Ultra"+"_APE"
            elif name == "HL2":
                return "Hololens2"+"_APE"
            elif name == "ML2":
                return "MagicLeap2"+"_APE"
            else:
                return name

In [13]:
# Calculate the RMS of RPE and APE columns
def calculate_column_rms(df_raw):
    # Calculate the RMS of RPE and APE columns
    rpe_columns = [col for col in df_raw.columns if "RPE" in col]
    ape_columns = [col for col in df_raw.columns if "APE" in col]

    # Calculate RMS for RPE
    rpe_rms = {}
    for col in rpe_columns:
        # Rename the column
        new_col = device_rename(col)
        rpe_rms[new_col] = np.sqrt(np.nanmean(df_raw[col]**2))

    # Calculate RMS for APE
    ape_rms = {}
    for col in ape_columns:
        # Rename the column
        new_col = device_rename(col)
        ape_rms[new_col] = np.sqrt(np.nanmean(df_raw[col]**2))

    return rpe_rms, ape_rms

# Calculate the mean and std of RPE and APE columns
def calculate_column_statistics(df_raw):
    # Calculate the RMS of RPE and APE columns
    rpe_columns = [col for col in df_raw.columns if "RPE" in col]
    ape_columns = [col for col in df_raw.columns if "APE" in col]

    # Calculate RMS for RPE
    rpe_mean = {}
    for col in rpe_columns:
        # Rename the column
        new_col = device_rename(col)
        rpe_mean[new_col] = np.nanmean(df_raw[col])
    rpe_std = {}
    for col in rpe_columns:
        # Rename the column
        new_col = device_rename(col)
        rpe_std[new_col] = np.nanstd(df_raw[col])

    # Calculate RMS for APE
    ape_mean = {}
    for col in ape_columns:
        # Rename the column
        new_col = device_rename(col)
        ape_mean[new_col] = np.nanmean(df_raw[col])
    ape_std = {}
    for col in ape_columns:
        # Rename the column
        new_col = device_rename(col)
        ape_std[new_col] = np.nanstd(df_raw[col])
    return rpe_mean, rpe_std, ape_mean, ape_std

In [16]:
import pandas as pd

import pandas as pd
import numpy as np

def sliding_window_smoothing(df, column_name, window_size):
    """
    Apply sliding window smoothing to a specified column in a pandas DataFrame,
    ensuring the result has the same length with no NaN values.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to smooth.
    window_size (int): The size of the sliding window.
    
    Returns:
    pd.Series: A smoothed copy of the specified column with the same length.
    """
    # Get the original column
    original = df[column_name].copy()
    
    # Create a temporary series with padding at the beginning
    # This allows us to calculate values for the beginning of the series
    padded = pd.Series(
        np.pad(original.values, (window_size-1, 0), mode='edge'),
        index=range(-(window_size-1), len(original))
    )
    
    # Apply rolling window and shift the result to align with original data
    smoothed = padded.rolling(window=window_size).mean().iloc[window_size-1:].reset_index(drop=True)
    
    # Ensure the index matches the original
    smoothed.index = original.index
    
    return smoothed


In [ ]:
# Calculate the RMS of RPE and APE columns from S1_raw and S2_raw
for motion in motions:
    for visual in visuals:
        for speed in speeds:
            df_combined_S1 = copy.deepcopy(datadict[motion][visual][speed]["S1_raw"])
            df_combined_S2 = copy.deepcopy(datadict[motion][visual][speed]["S2_raw"])
            df_trim_S1 = copy.deepcopy(datadict[motion][visual][speed]["S1_trim"] )
            df_trim_S2 = copy.deepcopy(datadict[motion][visual][speed]["S2_trim"] )

            # Calculate mean and std
            rpe_rms_S1, rpe_std_S1, ape_rms_S1, ape_std_S1 = calculate_column_statistics(df_trim_S1)
            rpe_rms_S2, rpe_std_S2, ape_rms_S2, ape_std_S2 = calculate_column_statistics(df_trim_S2)


            datadict[motion][visual][speed]["RPE_rmse"] = {}
            datadict[motion][visual][speed]["RPE_std"] = {}
            datadict[motion][visual][speed]["APE_rmse"] = {}
            datadict[motion][visual][speed]["APE_std"] = {}
            datadict[motion][visual][speed]["RPE"] = {}
            datadict[motion][visual][speed]["APE"] = {}


            datadict[motion][visual][speed]["S1_Sensor"] = {}
            datadict[motion][visual][speed]["S2_Sensor"] = {}


            # Load the following columns from the df_combined_S1 and df_combined_S2
            # Brightness Contrast Entropy Laplacian MatchedInlier NumberKeyPoints ACCx	ACCy	ACCz	AngVelx	AngVely	AngVelz
            # Save them in the datadict (Implement in a for-loop)
            sensor_columns = ["Brightness", "Contrast", "Entropy", "Laplacian", "MatchedInlier", "NumberKeyPoints",
                                "ACCx", "ACCy", "ACCz", "AngVelx", "AngVely", "AngVelz"]
            sensor_columns = ["Brightness", "Contrast", "Entropy", "Laplacian", "NumberKeyPoints",
                                "ACCx", "ACCy", "ACCz", "AngVelx", "AngVely", "AngVelz"]
            for col in sensor_columns:
                # Apply smoothing to the columns
                S1_col = sliding_window_smoothing(df_trim_S1, col, 5)
                S2_col = sliding_window_smoothing(df_trim_S2, col, 5)

                datadict[motion][visual][speed]["S1_Sensor"][col] = S1_col#.tolist()
                datadict[motion][visual][speed]["S2_Sensor"][col] = S2_col#.tolist()
            

            #Key is the device name
            for key in rpe_rms_S1.keys():
                datadict[motion][visual][speed]["RPE_rmse"][key] = rpe_rms_S1[key]
                datadict[motion][visual][speed]["RPE_std"][key] = rpe_std_S1[key]
                datadict[motion][visual][speed]["RPE"][key] = df_trim_S1[device_rename_reverse(key, ErrorType="RPE")].tolist()
            for key in ape_rms_S1.keys():
                datadict[motion][visual][speed]["APE_rmse"][key] = ape_rms_S1[key]
                datadict[motion][visual][speed]["APE_std"][key] = ape_std_S1[key]
                datadict[motion][visual][speed]["APE"][key] = df_trim_S1[device_rename_reverse(key,ErrorType="APE")].tolist()
                        
            # Key is the device name
            for key in rpe_rms_S2.keys():
                datadict[motion][visual][speed]["RPE_rmse"][key] = rpe_rms_S2[key]
                datadict[motion][visual][speed]["RPE_std"][key] = rpe_std_S2[key]
                datadict[motion][visual][speed]["RPE"][key] = df_trim_S2[device_rename_reverse(key, ErrorType="RPE")].tolist()
            for key in ape_rms_S2.keys():
                datadict[motion][visual][speed]["APE_rmse"][key] = ape_rms_S2[key]
                datadict[motion][visual][speed]["APE_std"][key] = ape_std_S2[key]
                datadict[motion][visual][speed]["APE"][key] = df_trim_S2[device_rename_reverse(key, ErrorType="APE")].tolist()    
            

In [ ]:
# For each device, 
# Compute the correlation between the RPE and sensor data

new_correlation_dict = {}

motions = ['Inspect', 'Patrol', 'Rotate', 'Shift']
visuals = ['Featureless', 'Featurerich']
speeds = ['50', '75']
devices = ["AVP", "MQ3", "ML2", "HL2", "XR2U", "ORB3"]

for device in devices:
    new_correlation_dict[device] = {"RPE": [], "Sensors": {}, "correlation": {}}
    for sensor in ["Brightness", "Contrast", "Entropy", "Laplacian", "NumberKeyPoints",
                                "ACCx", "ACCy", "ACCz", "AngVelx", "AngVely", "AngVelz"]:
        new_correlation_dict[device]["Sensors"][sensor] = []


for device in devices:
    for motion in motions:
        for visual in visuals:
            for speed in speeds:
                if device in ["MQ3", "XR2U"]:
                    sensor_dict = datadict[motion][visual][speed]["S1_Sensor"]
                elif device in ["AVP", "HL2", "ML2", "ORB3"]:
                    sensor_dict = datadict[motion][visual][speed]["S2_Sensor"]
                else:
                    continue

                # Concatenate all RPE data for the device
                RPE = datadict[motion][visual][speed]["RPE"][device]
                new_correlation_dict[device]["RPE"].append(RPE)

                # Concatenate all sensor data for the device
                for key in sensor_dict.keys():
                    sensor_data = sensor_dict[key]

                    # Use the absolute value of the following sensor data
                    if key in ["ACCx", "ACCy", "ACCz", "AngVelx", "AngVely", "AngVelz"]:
                        sensor_data = np.abs(sensor_data)
                    #print("Sensor data shape for {}: {}".format(key, np.shape(sensor_data)))
                    # Save the sensor data in the new_correlation_dict
                    new_correlation_dict[device]["Sensors"][key].append(sensor_data)

for device in devices:
    # Calculate the correlation between RPE and each sensor, then save it in the new_correlation_dict[device]["correlation"]
    for key in new_correlation_dict[device]["Sensors"].keys():
        # Concatenate all sensor data for the device
        #print("Device: {}, Sensor: {}".format(device, key))
        sensor_data = np.concatenate(new_correlation_dict[device]["Sensors"][key])
        RPE_data = np.concatenate(new_correlation_dict[device]["RPE"])
        # Calculate the correlation
        if len(RPE_data) > 0 and len(sensor_data) > 0:
            correlation = np.corrcoef(RPE_data, sensor_data)[0, 1]
            new_correlation_dict[device]["correlation"][key] = correlation
        else:
            new_correlation_dict[device]["correlation"][key] = np.nan

###################################################################################################

for device in devices:
    for motion in motions:
        for visual in visuals:
            for speed in speeds:
                # print df trim shape
                # print("S1 trim shape: {}".format(np.shape(datadict[motion][visual][speed]["S1_trim"])))
                # print("S2 trim shape: {}".format(np.shape(datadict[motion][visual][speed]["S2_trim"])))
                # print sensor shape
                # for sensor in sensor_columns:
                    # # print each sensor data shape
                    # print("S1 Sensor shape: {}".format(np.shape(datadict[motion][visual][speed]["S1_Sensor"][sensor])))
                    # print("S2 Sensor shape: {}".format(np.shape(datadict[motion][visual][speed]["S2_Sensor"][sensor])))
                    

                # if device in  "MQ3", "XR2U", use S1_Sensor
                # if device in "AVP", "HL2", "ML2", "ORB" use S2_Sensor
                if device in ["MQ3", "XR2U"]:
                    sensor_dict = datadict[motion][visual][speed]["S1_Sensor"]
                elif device in ["AVP", "HL2", "ML2", "ORB3"]:
                    sensor_dict = datadict[motion][visual][speed]["S2_Sensor"]
                else:
                    continue

                # Get the RPE data
                RPE = datadict[motion][visual][speed]["RPE"][device]

                # Calculate the correlation with each sensor
                correlation_dict = {}
                for key in sensor_dict.keys():
                    # Get the sensor data
                    sensor_data = sensor_dict[key]

                    # Use the absolute value of the following sensor data
                    if key in ["ACCx", "ACCy", "ACCz", "AngVelx", "AngVely", "AngVelz"]:
                        sensor_data = np.abs(sensor_data)

                    #print("Sensor data shape: {}".format(np.shape(sensor_data)))
                    # Calculate the correlation ()
                    correlation = np.corrcoef(RPE, sensor_data)[0, 1]
                    # Save the correlation
                    correlation_dict[key] = correlation

                # Print the correlation dict for the AVP device
                if device == "AVP":
                    print("Motion: {}, Visual: {}, Speed: {}".format(motion, visual, speed))
                    print("Correlation dict for {}: {}".format(device, correlation_dict['NumberKeyPoints']))


                # Save the device's correlation dict in datadict
                datadict[motion][visual][speed]["Correlation"][device] = correlation_dict


Motion: Inspect, Visual: Featureless, Speed: 50
Correlation dict for AVP: 0.13952248062923786
Motion: Inspect, Visual: Featureless, Speed: 75
Correlation dict for AVP: 0.056380438682074985
Motion: Inspect, Visual: Featurerich, Speed: 50
Correlation dict for AVP: -0.010690492921711247
Motion: Inspect, Visual: Featurerich, Speed: 75
Correlation dict for AVP: -0.05136892665037672
Motion: Patrol, Visual: Featureless, Speed: 50
Correlation dict for AVP: -0.06616033979181628
Motion: Patrol, Visual: Featureless, Speed: 75
Correlation dict for AVP: -0.06955606786834492
Motion: Patrol, Visual: Featurerich, Speed: 50
Correlation dict for AVP: -0.04813955119498727
Motion: Patrol, Visual: Featurerich, Speed: 75
Correlation dict for AVP: 0.004943219835926388
Motion: Rotate, Visual: Featureless, Speed: 50
Correlation dict for AVP: 0.22902881852532272
Motion: Rotate, Visual: Featureless, Speed: 75
Correlation dict for AVP: -0.03434496146518775
Motion: Rotate, Visual: Featurerich, Speed: 50
Correlatio

In [20]:
new_correlation_dict['AVP']['correlation']

{'Brightness': np.float64(0.001845930444514111),
 'Contrast': np.float64(0.027414345835504936),
 'Entropy': np.float64(-0.0423507218720033),
 'Laplacian': np.float64(0.03588204685122719),
 'NumberKeyPoints': np.float64(0.056558144521444434),
 'ACCx': np.float64(0.07963535632880266),
 'ACCy': np.float64(-0.032636071365427455),
 'ACCz': np.float64(0.10340939014325241),
 'AngVelx': np.float64(0.14104311922636903),
 'AngVely': np.float64(-0.031138089526430168),
 'AngVelz': np.float64(0.09712679860824333)}

In [21]:
avg_correlation_dict = {}
for device in devices:
    for motion in motions:
        for visual in visuals:
            for speed in speeds:
                # Get the correlation dict
                print("Device: {}, Motion: {}, Visual: {}, Speed: {}".format(device, motion, visual, speed))
                correlation_dict = datadict[motion][visual][speed]["Correlation"][device]
                # Compute the average correlation 
                for key in correlation_dict.keys():
                    # for each sensor key, append the correlation value to the avg_correlation_dict
                    if key not in avg_correlation_dict:
                        avg_correlation_dict[key] = []
                    avg_correlation_dict[key].append(correlation_dict[key])
    # # Compute the average

Device: AVP, Motion: Inspect, Visual: Featureless, Speed: 50
Device: AVP, Motion: Inspect, Visual: Featureless, Speed: 75
Device: AVP, Motion: Inspect, Visual: Featurerich, Speed: 50
Device: AVP, Motion: Inspect, Visual: Featurerich, Speed: 75
Device: AVP, Motion: Patrol, Visual: Featureless, Speed: 50
Device: AVP, Motion: Patrol, Visual: Featureless, Speed: 75
Device: AVP, Motion: Patrol, Visual: Featurerich, Speed: 50
Device: AVP, Motion: Patrol, Visual: Featurerich, Speed: 75
Device: AVP, Motion: Rotate, Visual: Featureless, Speed: 50
Device: AVP, Motion: Rotate, Visual: Featureless, Speed: 75
Device: AVP, Motion: Rotate, Visual: Featurerich, Speed: 50
Device: AVP, Motion: Rotate, Visual: Featurerich, Speed: 75
Device: AVP, Motion: Shift, Visual: Featureless, Speed: 50
Device: AVP, Motion: Shift, Visual: Featureless, Speed: 75
Device: AVP, Motion: Shift, Visual: Featurerich, Speed: 50
Device: AVP, Motion: Shift, Visual: Featurerich, Speed: 75
Device: MQ3, Motion: Inspect, Visual: Fe

In [24]:
average_list = []
sensors = list(avg_correlation_dict.keys())
device = "HL2"
for motion in motions:
    for visual in visuals:
        for speed in speeds:
            correlations = [datadict[motion][visual][speed]["Correlation"][device][sensor] 
                            for sensor in sensors]
            average_list.append(correlations)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Define the devices and sensors
devices = ["AVP", "MQ3", "ML2", "HL2","XR2U"]#, "ORB3"]
sensors = list(avg_correlation_dict.keys())

# Set font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'

# Define colors and hatches for devices
colors = ['#7f7f7f', '#bfbfbf', '#d9d9d9', '#f0f0f0', '#ffffff', '#000000']
hatches =  ['***', '...', '///', '---', '\\\\', 'xxx']

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 3))

# Define the bar width and positions
bar_width = 0.15
x = np.arange(len(sensors))

for i, device in enumerate(devices):
    average_list = []
    for motion in motions:
        # if motion != "Rotate":
        #     continue
        for visual in visuals:
            for speed in speeds:
                correlations = [datadict[motion][visual][speed]["Correlation"][device][sensor] 
                                for sensor in sensors]
                average_list.append(correlations)


    average_array = np.array(average_list)
    print("Average array shape: {}".format(np.shape(average_array)))
    # Calculate the mean correlation for each sensor
    correlations = np.mean(average_array, axis=0)

    print("Correlations shape: {}".format(np.shape(correlations)))
    ax.bar(x + i * bar_width, correlations, bar_width, label=device, color=colors[i], hatch=hatches[i], edgecolor='black')


# Add labels, title, and legend
ax.set_xlabel('Sensors', fontsize=14)
ax.set_ylabel('Correlation', fontsize=14)
ax.set_facecolor('none')  # Makes the plot background transparent
ax.grid(True, axis='y', linestyle='--', alpha=0.7, color='#444444')  # Adds a dashed grid 
ax.set_title('Correlation of RPE and Sensors for All Devices', fontsize=14)
ax.set_xticks(x + bar_width * (len(devices) + 4) / 2)
sensors = [sensor.replace("NumberKeyPoints", "Number\nKeypoints") for sensor in sensors]  # Replace NumberKeyPoints with Keypoints
sensors = [sensor.replace("ACCx", "Acc\nRight") for sensor in sensors]  # Replace ACCx with ACC\nX
sensors = [sensor.replace("ACCy", "Acc\nUp") for sensor in sensors]  # Replace ACCy with ACC\nY
sensors = [sensor.replace("ACCz", "Acc\nFront") for sensor in sensors]  # Replace ACCz with ACC\nZ
sensors = [sensor.replace("AngVelx", "AngVel\nPitch") for sensor in sensors]  # Replace AngVelx with AngVel\nPitch
sensors = [sensor.replace("AngVely", "AngVel\nYaw") for sensor in sensors]  # Replace AngVely with AngVel\nYaw
sensors = [sensor.replace("AngVelz", "AngVel\nRoll") for sensor in sensors]  # Replace AngVelz with AngVel\nRoll

plt.tight_layout()

ax.set_xticklabels(sensors, rotation=0, ha='right', fontdict={'fontsize': 12})
ax.tick_params(axis='x', which='major', tick1On=False, tick2On=False, pad=-5)

# Show the plot
plt.show()

Average array shape: (16, 11)
Correlations shape: (11,)
Average array shape: (16, 11)
Correlations shape: (11,)
Average array shape: (16, 11)
Correlations shape: (11,)
Average array shape: (16, 11)
Correlations shape: (11,)
Average array shape: (16, 11)
Correlations shape: (11,)


/tmp/ipykernel_795601/2378245975.py:76: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [29]:
# Calculate the average RMS of RPE and APE for each device across all trajectories
def calculate_average_rms(datadict):
    average_rpe_rmse = {}
    average_ape_rmse = {}
    for motion in motions:
        for visual in visuals:
            for speed in speeds:
                # Get the RPE and APE rmse
                rpe_rmse = datadict[motion][visual][speed]["RPE_rmse"]
                ape_rmse = datadict[motion][visual][speed]["APE_rmse"]

                # Calculate the average
                for key in rpe_rmse.keys():
                    if key not in average_rpe_rmse:
                        average_rpe_rmse[key] = []
                    average_rpe_rmse[key].append(rpe_rmse[key])

                for key in ape_rmse.keys():
                    if key not in average_ape_rmse:
                        average_ape_rmse[key] = []
                    average_ape_rmse[key].append(ape_rmse[key])

    # Calculate the average
    for key in average_rpe_rmse.keys():
        average_rpe_rmse[key] = np.mean(average_rpe_rmse[key])
    for key in average_ape_rmse.keys():
        average_ape_rmse[key] = np.mean(average_ape_rmse[key])

    return average_rpe_rmse, average_ape_rmse


In [30]:
average_rpe_rmse, average_ape_rmse = calculate_average_rms(datadict)
average_ape_rmse

{'ORB3': np.float64(0.06625722668984985),
 'AVP': np.float64(0.0362100171699197),
 'MQ3': np.float64(0.04520616520021088),
 'XR2U': np.float64(0.0844066103354529),
 'HL2': np.float64(0.0910755140192199),
 'ML2': np.float64(0.06111638270824596)}

In [31]:
# Given the average rms, 
# calculate the percentage of increase/decrease using AVP as baseline
def calculate_percentage_change(average_rms):
    percentage_change = {}
    for key in average_rms.keys():
        if key == "AVP":
            percentage_change[key] = 0
        else:
            percentage_change[key] = (average_rms[key] - average_rms["AVP"]) / average_rms["AVP"] * 100
    return percentage_change

In [32]:
rpe_percentage = calculate_percentage_change(average_rpe_rmse)
rpe_percentage

{'ORB3': np.float64(195.84601868265756),
 'AVP': 0,
 'MQ3': np.float64(48.71504332243368),
 'XR2U': np.float64(150.87392342408094),
 'HL2': np.float64(178.0890390745125),
 'ML2': np.float64(79.65572101577649)}

In [33]:
ape_percentage = calculate_percentage_change(average_ape_rmse)
ape_percentage

{'ORB3': np.float64(82.98037910042996),
 'AVP': 0,
 'MQ3': np.float64(24.84436278523624),
 'XR2U': np.float64(133.1029282294043),
 'HL2': np.float64(151.52021771168322),
 'ML2': np.float64(68.78308127126881)}

In [34]:
# Given the average rms and the percentage change,
# generate a table in latex
# Convert the data to the format required for the table

# Create a mapping between device names in the data and in the table
device_mapping = {
    'XR2U': 'XR2U',
    'HL2': 'HL2',
    'ML2': 'ML2',
    'MQ3': 'MQ3',
    'AVP': 'AVP',
    'ORB3': 'ORB3'  # This one isn't in the table example
}

table_data = {}
for device_code, device_name in device_mapping.items():
    if device_code in average_rpe_rmse:
        # Convert meters to centimeters and round to 2 decimal places
        rpe_cm = round(average_rpe_rmse[device_code] * 100, 2)
        rpe_percent = round(rpe_percentage[device_code], 1)
        ape_cm = round(average_ape_rmse[device_code] * 100, 2)
        ape_percent = round(ape_percentage[device_code], 1)
        
        table_data[device_name] = {
            'rpe_cm': rpe_cm,
            'rpe_percent': rpe_percent,
            'ape_cm': ape_cm,
            'ape_percent': ape_percent
        }

# Generate the updated LaTeX table
latex_table = r"""\begin{table}[th!]
\centering
\label{tab: OverallTrackingResult}
\caption{Overall Tracking result}
\begin{tabular}{lllll}
\textbf{\begin{tabular}[c]{@{}l@{}}XR\\ Devices\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}RPE\\ (cm)\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}RPE+\\ w.r.t AVP\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}APE\\ (cm)\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}APE+\\ w.r.t AVP\end{tabular}} \\ \hline"""

# Order of devices in the table
device_order = ['ORB3','XR2U', 'HL2', 'ML2', 'MQ3', 'AVP']

for device in device_order:
    if device in table_data:
        data = table_data[device]
        latex_table += f"\n{device} & {data['rpe_cm']} & {data['rpe_percent']}\\% & {data['ape_cm']} & {data['ape_percent']}\\%"
        #if device 
    else:
        latex_table += f"\n{device} & ? & ?\\% & ? & ?\\%"
    
    latex_table += " \\\\"

latex_table += r"""
\end{tabular}
\end{table}"""

print(latex_table)

\begin{table}[th!]
\centering
\label{tab: OverallTrackingResult}
\caption{Overall Tracking result}
\begin{tabular}{lllll}
\textbf{\begin{tabular}[c]{@{}l@{}}XR\\ Devices\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}RPE\\ (cm)\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}RPE+\\ w.r.t AVP\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}APE\\ (cm)\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}l@{}}APE+\\ w.r.t AVP\end{tabular}} \\ \hline
ORB3 & 1.53 & 195.8\% & 6.63 & 83.0\% \\
XR2U & 1.29 & 150.9\% & 8.44 & 133.1\% \\
HL2 & 1.43 & 178.1\% & 9.11 & 151.5\% \\
ML2 & 0.93 & 79.7\% & 6.11 & 68.8\% \\
MQ3 & 0.77 & 48.7\% & 4.52 & 24.8\% \\
AVP & 0.52 & 0\% & 3.62 & 0\% \\
\end{tabular}
\end{table}


In [37]:

def plot_average_rpe_ape(datadict):
    # Create a figure and axis
    fig, ax = plt.subplots(2, 1, figsize=(10, 10))

    # Set the title
    fig.suptitle('Average RPE and APE for all combinations of motion, visual, and speed')

    # Iterate over the datadict and plot the data
    for motion in motions:
        for visual in visuals:
            for speed in speeds:
                # Get the data
                rpe_rmse = datadict[motion][visual][speed]["RPE_rmse"]
                ape_rmse = datadict[motion][visual][speed]["APE_rmse"]

                # Plot the RPE data
                ax[0].bar(rpe_rmse.keys(), rpe_rmse.values(), label='{}-{}-{}'.format(motion, visual, speed))
                ax[1].bar(ape_rmse.keys(), ape_rmse.values(), label='{}-{}-{}'.format(motion, visual, speed))

    # Set the labels and title for RPE
    ax[0].set_ylabel('RPE (cm)')
    ax[0].set_title('Average RPE')
    ax[0].legend()

    # Set the labels and title for APE
    ax[1].set_ylabel('APE (cm)')
    ax[1].set_title('Average APE')
    ax[1].legend()

    # Show the plot
    plt.show()
#plot_average_rpe_ape(datadict)

In [38]:
motions = ['Rotate', 'Shift', 'Inspect', 'Patrol']
visuals = ['Featureless', 'Featurerich']
visual_ids = {'Featureless': 'FL', 'Featurerich': 'FR'}  # Shortened visual IDs for the table

# Start the LaTeX table
latex_table = r"""\begin{table}[h!]
\caption{Experiment design with different motions and visuals.}
\label{tab:experimentSetting}
\begin{tabular}{|c|c|c|c|}
\hline
\textbf{Motion} & \textbf{Visual} & \textbf{Ex.ID} & \textbf{Note} \\ \hline
"""

# Generate rows for the table
for motion in motions:
    note = f"Note for {motion}"  # Example note for each motion
    for i, visual in enumerate(visuals):
        ex_id = f"{motion[0]}-{visual_ids[visual]}"  # Generate Ex.ID
        if i == 0:
            # Merge the first column for the motion
            latex_table += fr"\multirow{{{len(visuals)}}}{{*}}{{\textbf{{{motion}}}}} & {visual} & {ex_id} & \multirow{{{len(visuals)}}}{{*}}{{{note}}} \\ \cline{{2-3}}" + "\n"
        else:
            # Add subsequent rows for the same motion
            latex_table += fr" & {visual} & {ex_id} & \\ \cline{{2-3}}" + "\n"
    
    # Add an extra \hline after each motion group
    latex_table += r"\hline" + "\n"

# End the LaTeX table - fixed the multi-line string issue
latex_table += r"\end{tabular}" + "\n"
latex_table += r"\vspace{-0.6cm}" + "\n"
latex_table += r"\end{table}"

print(latex_table)


\begin{table}[h!]
\caption{Experiment design with different motions and visuals.}
\label{tab:experimentSetting}
\begin{tabular}{|c|c|c|c|}
\hline
\textbf{Motion} & \textbf{Visual} & \textbf{Ex.ID} & \textbf{Note} \\ \hline
\multirow{2}{*}{\textbf{Rotate}} & Featureless & R-FL & \multirow{2}{*}{Note for Rotate} \\ \cline{2-3}
 & Featurerich & R-FR & \\ \cline{2-3}
\hline
\multirow{2}{*}{\textbf{Shift}} & Featureless & S-FL & \multirow{2}{*}{Note for Shift} \\ \cline{2-3}
 & Featurerich & S-FR & \\ \cline{2-3}
\hline
\multirow{2}{*}{\textbf{Inspect}} & Featureless & I-FL & \multirow{2}{*}{Note for Inspect} \\ \cline{2-3}
 & Featurerich & I-FR & \\ \cline{2-3}
\hline
\multirow{2}{*}{\textbf{Patrol}} & Featureless & P-FL & \multirow{2}{*}{Note for Patrol} \\ \cline{2-3}
 & Featurerich & P-FR & \\ \cline{2-3}
\hline
\end{tabular}
\vspace{-0.6cm}
\end{table}


In [39]:
def generate_latex_table(datadict, motions, visuals, speeds, devices):
    latex_table = r"""\begin{table*}[h!]
\centering
\begin{tabular}{|c|c|c|"""
    
    latex_table += "c|" * len(devices) * 2 + r"}"
    latex_table += r"\hline" + "\n"
    
    # Header row - all bold
    latex_table += r"\textbf{Motion} & \textbf{Visual} & \textbf{Speed}"
    for device in devices:
        latex_table += fr" & \begin{{tabular}}[c]{{@{{}}c@{{}}}} \textbf{{{device}}} \\ \textbf{{RPE}} \end{{tabular}}"
    for device in devices:
        latex_table += fr" & \begin{{tabular}}[c]{{@{{}}c@{{}}}} \textbf{{{device}}} \\ \textbf{{APE}} \end{{tabular}}"
    latex_table += r" \\" + "\n" + r"\hline" + "\n"
    
    total_columns = 3 + len(devices) * 2  # Calculate total number of columns
    
    for motion in motions:
        first_motion_row = True
        for visual in visuals:
            first_visual_row = True
            for speed in speeds:
                row = ""
                if first_motion_row:
                    row += fr"\multirow{{8}}{{*}}{{\textbf{{{motion}}}}} & "
                    first_motion_row = False
                else:
                    row += r" & "
                
                if first_visual_row:
                    row += fr"\multirow{{2}}{{*}}{{\textbf{{{visual}}}}} & "
                    first_visual_row = False
                else:
                    row += r" & "
                
                row += fr"\textbf{{{speed}}}"
                
                for device in devices:
                    rpe_value = datadict.get(motion, {}).get(visual, {}).get(speed, {}).get("RPE_rmse", {}).get(device, "-")
                    if rpe_value != "-":
                        rpe_value = f"{float(rpe_value) * 100:.2f}"  # Convert to cm and format
                    row += f" & {rpe_value}"
                
                for device in devices:
                    ape_value = datadict.get(motion, {}).get(visual, {}).get(speed, {}).get("APE_rmse", {}).get(device, "-")
                    if ape_value != "-":
                        ape_value = f"{float(ape_value) * 100:.2f}"  # Convert to cm and format
                    row += f" & {ape_value}"
                
                row += r" \\" + "\n"
                latex_table += row
            
            if visual != visuals[-1]:
                latex_table += fr"\cline{{2-{total_columns}}}" + "\n"
        
        if motion != motions[-1]:
            latex_table += r"\hline" + "\n"
    
    latex_table += r"""\hline
\end{tabular}
\caption{RPE and APE RMSE values (in cm) for different motions, visuals, and speeds.}
\label{tab:rmse_values}
\end{table*}"""
    
    return latex_table


In [40]:
devices = ["AVP", "MQ3","XR2U", "HL2", "ML2", "ORB3"]
devices = ["AVP", "MQ3", "ML2", "HL2", "XR2U", "ORB3"]
visuals = ['Featureless', 'Featurerich']
result = generate_latex_table(datadict, motions, visuals, speeds, devices)
print(result)

\begin{table*}[h!]
\centering
\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}\hline
\textbf{Motion} & \textbf{Visual} & \textbf{Speed} & \begin{tabular}[c]{@{}c@{}} \textbf{AVP} \\ \textbf{RPE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{MQ3} \\ \textbf{RPE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{ML2} \\ \textbf{RPE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{HL2} \\ \textbf{RPE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{XR2U} \\ \textbf{RPE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{ORB3} \\ \textbf{RPE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{AVP} \\ \textbf{APE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{MQ3} \\ \textbf{APE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{ML2} \\ \textbf{APE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{HL2} \\ \textbf{APE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{XR2U} \\ \textbf{APE} \end{tabular} & \begin{tabular}[c]{@{}c@{}} \textbf{ORB3} \\ \textbf{APE

In [43]:
import matplotlib.pyplot as plt
import numpy as np

# Define the categories
motions = ['Inspect', 'Patrol', 'Rotate', 'Shift']
visuals = ['Featureless', 'Featurerich']
visual_ids = {'Featureless': 'FL', 'Featurerich': 'FR'}  # Shortened visual IDs for the table
speeds = ['50', '75']
devices = ["AVP", "MQ3", "XR2U", "HL2", "ML2", "ORB3"]
devices = ["AVP", "MQ3", "ML2", "HL2", "XR2U", "ORB3"]

# Use grayscales for better visibility
colors = ['#7f7f7f', '#bfbfbf', '#d9d9d9', '#f0f0f0', '#ffffff', '#000000']
hatches = ['***', '...', '///', '---', '\\\\', 'xxx']

# Set font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'

# Define figsize and dpi for all figures
figsize = (2.5, 1.5)
dpi = 200

# Set ranges for RPE and APE
rpe_range = (0,9)  # Adjust as needed
ape_range = (0, 32)  # Adjust as needed
rpe_range = (0,9)  # Adjust as needed
ape_range = (0, 39)  # Adjust as needed

# Function to create bar charts for a motion
def create_bar_charts(datadict, motion, visuals, speeds, devices):
    if motion not in datadict:
        return
        
    # Prepare data for RPE and APE
    rpe_values = []
    ape_values = []
    rpe_stds = []
    ape_stds = []
    labels = []
    
    # Collect data for all visual-speed combinations
    for visual in visuals:
        if visual not in datadict[motion]:
            continue
            
        for speed in speeds:
            if speed not in datadict[motion][visual]:
                continue
                
            # Create label for this combination
            label = f"{motion[0]}-{visual_ids[visual]}-{speed}"
            labels.append(label)
            
            # Add RPE and APE values and stds
            rpe_values.append([datadict[motion][visual][speed]['RPE_rmse'].get(device, 0) for device in devices])
            ape_values.append([datadict[motion][visual][speed]['APE_rmse'].get(device, 0) for device in devices])
            rpe_stds.append([datadict[motion][visual][speed]['RPE_std'].get(device, 0) for device in devices])
            ape_stds.append([datadict[motion][visual][speed]['APE_std'].get(device, 0) for device in devices])
    
    # Convert to numpy arrays for plotting (meter to cm conversion)
    rpe_values = np.array(rpe_values) * 100.0
    ape_values = np.array(ape_values) * 100.0
    
    rpe_stds = np.array(rpe_stds) * 100.0
    ape_stds = np.array(ape_stds) * 100.0

    ape_stds[ape_values > 25] = 0
    rpe_stds[ape_values > 25] = 0
    rpe_values[ape_values > 25] = 0
    ape_values[ape_values > 25] = 0
    

    rpe_stds[rpe_values > 7] = 0
    ape_stds[rpe_values > 7] = 0
    ape_values[rpe_values > 7] = 0
    rpe_values[rpe_values > 7] = 0
    
    # Create x positions for bars
    x = np.arange(len(labels))
    bar_width = 0.12
    
    # Create RPE bar chart
    plt.figure(figsize=figsize, dpi=dpi)
    ax = plt.axes()
    ax.set_facecolor('none')
    max_y = 0
    for i, device in enumerate(devices):
        plt.bar(x + i * bar_width, rpe_values[:, i], bar_width, 
                yerr=rpe_stds[:, i], capsize=1.5, error_kw={'elinewidth':1, 'capthick':1, 'ecolor': "grey"}, # error bars added
                label=device, color=colors[i], hatch=hatches[i], edgecolor='black')
        _max_y = rpe_values[:, i]+rpe_stds[:, i]
        if max_y < _max_y.max():
            max_y = _max_y.max()
    
    # Add labels, title, and legend
    plt.xlabel('Experiment ID', fontsize=9, labelpad=1)
    plt.ylabel('RPE (cm)', fontsize=9, labelpad=1)
    plt.title(f'RPE for {motion} motion', fontsize=10, fontweight='bold', y=0.9)
    plt.xticks(x + bar_width * (len(devices)+6) / 2, labels, rotation=0, ha='right', fontsize=9)
    #plt.ylim(rpe_range)
    ax.tick_params(axis='x', which='major', tick1On=False, tick2On=False,pad=-5, labelsize=8)
    #ax.set_yticks(np.arange(0, 9, 2))
    if max_y < 5:
        ax.set_yticks(np.arange(0, max_y, 1))
    else:
        ax.set_yticks(np.arange(0, max_y, 2))
    ax.tick_params(axis='y', which='major', tick1On=False, tick2On=False, pad=-5, labelsize=9)

    # legend
    #plt.legend(title='Devices', fontsize=8, loc='upper left', bbox_to_anchor=(1, 1))
    plt.grid(axis='y', linestyle='--', alpha=0.2, color='gray')
    plt.tight_layout()

    plt.savefig(f'{motion}_RPE.png', bbox_inches='tight', pad_inches=0.02)
    plt.show()
    
    # Create APE bar chart
    plt.figure(figsize=figsize, dpi=dpi)
    ax = plt.axes()
    ax.set_facecolor('none')
    max_y = 0
    for i, device in enumerate(devices):
        plt.bar(x + i * bar_width, ape_values[:, i], bar_width, 
                yerr=ape_stds[:, i], capsize=1.5, error_kw={'elinewidth':1, 'capthick':1, 'ecolor': "grey"},  # error bars added
                label=device, color=colors[i], hatch=hatches[i], edgecolor='black')
        _max_y = ape_values[:, i]+ape_stds[:, i]
        if max_y < _max_y.max():
            max_y = _max_y.max()
    
    # Add labels, title, and legend
    plt.xlabel('Experiment ID', fontsize=9, labelpad=1)
    plt.ylabel('APE (cm)',fontsize=9, labelpad=1)
    plt.title(f'APE for {motion} motion', fontsize=10, fontweight='bold', y=0.9)
    plt.xticks(x + bar_width * (len(devices)+6) / 2, labels, rotation=0, ha='right', fontsize=9)
    ax.tick_params(axis='x', which='major', tick1On=False, tick2On=False, pad=-5, labelsize=8)
    #ax.set_yticks(np.arange(0, 35, 5))
    #ax.set_yticks(np.arange(0, max_y, 5))
    if max_y < 15:
        ax.set_yticks(np.arange(0, max_y, 3))
    else:
        ax.set_yticks(np.arange(0, max_y, 5))
    
    ax.tick_params(axis='y', which='major', tick1On=False, tick2On=False, pad=-5, labelsize=9)
    #plt.ylim(ape_range)
    plt.grid(axis='y', linestyle='--', alpha=0.2, color='gray')
    plt.tight_layout()
    plt.savefig(f'{motion}_APE.png', bbox_inches='tight', pad_inches=0.02)
    plt.show()

# Generate bar charts for each motion
for motion in motions:
    create_bar_charts(datadict, motion, visuals, speeds, devices)
    #break

/tmp/ipykernel_795601/3663806898.py:112: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:146: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:112: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:146: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:112: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:146: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:112: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_795601/3663806898.py:146: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
